In [ ]:
from typing import List
import nest_asyncio
import logging
import os
import streamlit as st

from phi.assistant import Assistant
from phi.document import Document
from phi.document.reader.pdf import PDFReader

from phi.knowledge import AssistantKnowledge
from phi.tools.duckduckgo import DuckDuckGo
from langchain_community.vectorstores import FAISS
from langchain.llms import GooglePalm
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.document_loaders import PyPDFLoader

# Set Google API key from environment variables
os.environ['GOOGLE_API_KEY'] = "GOOGLE_API_KEY"

# Create Google Palm LLM model
llm = GooglePalm(temperature=0.5)

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")
vectordb_file_path = "faiss_vector_database"

logger = logging.getLogger(__name__)

# Function to create vector database with FAISS
def create_vector_db(file_path: str):
    loader = PyPDFLoader(file_path)
    data = loader.load()
    vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)
    vectordb.save_local(vectordb_file_path)
    return vectordb

# Function to load the vector database
def load_vector_db():
    if not os.path.exists(vectordb_file_path):
        raise FileNotFoundError(f"FAISS vector database file '{vectordb_file_path}' does not exist.")
    return FAISS.load_local(vectordb_file_path, instructor_embeddings)

# Function to add a document to the knowledge base
def add_document_to_kb(file_path: str):
    vectordb = create_vector_db(file_path)
    logger.info(f"Document '{file_path}' added to the knowledge base.")



if __name__ == "__main__":
    nest_asyncio.apply()
    sample_pdf_path = "file.pdf"
    add_document_to_kb(sample_pdf_path)
